In [22]:
# Update der Datensätze in der Tabelle dbo.broccolivalues -> Erweiterung mit Erntedaten
%reset
import pyodbc
import pandas as pd
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import numpy as np

# CSV einlesen und Datensätze filtern, die in DB geschrieben werden sollen
#filename = ''
#cropValues = pd.read_csv(filename, delimiter=';')

# Azure-DB Verbindungsaufbau
server = 'deepbroccoliserver.database.windows.net'
database = 'DeepBroccoliDatabase'
username = 'ntb'
password = 'brokkoli_2019'
driver= '{SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)

# Abfrage der Brokkoli-Daten aus dbo.broccoli
query = 'SELECT * FROM dbo.broccoli'
SQL_Query = pd.read_sql_query(query, conn)
broccoli_data = pd.DataFrame(SQL_Query)

coords = []
for index, row in broccoli_data.iterrows():
    coords.append(Point(row['lat'], row['long']))
    
broccoli_data['coord'] = coords
broccoli_data

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


,id,lat,long,cropMissing,cropOverripe,cropRudimentary,cropRotten,cropUnripe,cropWeight,cropNoBlossom,rowNumber,coord
0,1,47.217185,9.492887,False,False,True,False,False,NaN,False,None,POINT (47.21718464 9.492887284)
1,2,47.217187,9.492902,False,False,False,False,True,NaN,False,None,POINT (47.21718655 9.492901635999999)
2,3,47.217186,9.492892,False,False,False,True,False,NaN,False,None,POINT (47.217186 9.492892438)
3,4,47.217191,9.492904,False,False,False,False,False,504.0,False,None,POINT (47.21719056 9.492903630000001)
4,5,47.217190,9.492893,False,False,False,False,True,171.0,False,None,POINT (47.2171896 9.492892506)
5,6,47.217194,9.492895,False,False,False,False,False,515.0,False,None,POINT (47.21719358 9.492895280000001)
6,7,47.217193,9.492905,False,False,False,False,False,306.0,False,None,POINT (47.21719342 9.492905485)
7,8,47.217196,9.492908,False,False,False,False,False,539.0,False,None,POINT (47.21719618 9.492907670999999)
8,9,47.217197,9.492897,True,False,False,False,False,NaN,False,None,POINT (47.21719733 9.492897336)
9,10,47.217200,9.492899,False,False,False,False,False,525.0,False,None,POINT (47.21720001 9.492899415)


In [23]:
filename = 'Zusammenfügen_von_Koordinaten_Ernte.csv'
cropValues = pd.read_csv(filename, delimiter=';')

coords = []
for index, row in cropValues.iterrows():
    coords.append(Point(row['latitude'], row['longitude']))
    
cropValues['coord'] = coords
cropValues['sqlID'] = 0

In [24]:
coordsMultiPoint = MultiPoint(broccoli_data['coord'])
for index, row in cropValues.iterrows():
    coordCrop = row['coord']
    nearest_geoms = nearest_points(coordCrop, coordsMultiPoint)
    coord = nearest_geoms[1]
    broccoli = broccoli_data.loc[(broccoli_data['lat'] == coord.x) & (broccoli_data['long'] == coord.y)]
    cropValues.at[index, 'sqlID'] = broccoli['id']
    
cropValues

,id_measurement,latitude,longitude,id_ernte,fehlt,ueberreif,verkuemmert,keinKopf,faul,unreif,Gewicht,coord,sqlID
0,Hofmaenner_F_001.001,47.218537,9.493662,Hofmaenner_F_001.001,False,False,False,False,False,False,490,POINT (47.2185367 9.493662494000001),2566
1,Hofmaenner_F_001.002,47.218533,9.493660,Hofmaenner_F_001.002,False,False,False,False,False,False,520,POINT (47.21853346 9.493660373999999),2560
2,Hofmaenner_F_001.003,47.218530,9.493659,Hofmaenner_F_001.003,False,False,False,False,False,False,515,POINT (47.2185305 9.49365884),2556
3,Hofmaenner_F_001.004,47.218527,9.493657,Hofmaenner_F_001.004,False,False,False,False,False,False,562,POINT (47.21852749 9.493657413999999),2550
4,Hofmaenner_F_001.005,47.218524,9.493655,Hofmaenner_F_001.005,False,False,False,False,False,False,431,POINT (47.21852401 9.493655409),2548
5,Hofmaenner_F_001.006,47.218522,9.493654,Hofmaenner_F_001.006,False,False,False,False,False,False,790,POINT (47.21852167 9.493653867999999),2538
6,Hofmaenner_F_001.007,47.218519,9.493652,Hofmaenner_F_001.007,False,False,False,False,False,False,392,POINT (47.21851861 9.49365216),2533
7,Hofmaenner_F_001.008,47.218516,9.493651,Hofmaenner_F_001.008,False,False,False,False,False,False,491,POINT (47.2185157 9.493650618),2527
8,Hofmaenner_F_001.009,47.218512,9.493649,Hofmaenner_F_001.009,False,False,False,False,False,False,766,POINT (47.21851213 9.493648734000001),2516
9,Hofmaenner_F_001.010,47.218509,9.493647,Hofmaenner_F_001.010,False,False,False,False,False,False,575,POINT (47.21850939 9.493646902),2515


In [25]:
print(len(cropValues))
cropValues.drop_duplicates(['sqlID'], keep=False, inplace=True)
print(len(cropValues))
duplicates = cropValues[cropValues.duplicated(['sqlID'])]
print(len(duplicates))

2681
2675
0


In [26]:
cropValues.fehlt = cropValues.fehlt.astype(int)
cropValues.ueberreif = cropValues.ueberreif.astype(int)
cropValues.verkuemmert = cropValues.verkuemmert.astype(int)
cropValues.keinKopf = cropValues.keinKopf.astype(int)
cropValues.faul = cropValues.faul.astype(int)
cropValues.unreif = cropValues.unreif.astype(int)

In [27]:
# UPDATE
cursor = conn.cursor()
for index, row in cropValues.iterrows():    
    # Update crop Labels
    #updateStatement = "Update dbo.broccoli set CropMissing=?, cropOverripe=?, cropRudimentary=?, cropRotten=?, cropUnripe=?, 
    #cropNoBlossom=?, cropWeight=? where dbo.broccoli.id=?"
    #values = (row['fehlt'], row['ueberreif'], row['verkuemmert'], row['faul'], row['unreif'], row['keinKopf'], row['Gewicht'], 
    #row['sqlID'])     
   
    # Update rowNumber
    updateStatement = "Update dbo.broccoli set rowNumber=? where dbo.broccoli.id=?"
    values = (int(row['id_measurement'][13:16]), row['sqlID'])     
    print(values)
    cursor.execute(updateStatement, values)

conn.commit()
conn.close()

(1, 2566)
(1, 2560)
(1, 2556)
(1, 2550)
(1, 2548)
(1, 2538)
(1, 2533)
(1, 2527)
(1, 2516)
(1, 2515)
(1, 2511)
(1, 2502)
(1, 2494)
(1, 2483)
(1, 2479)
(1, 2476)
(1, 2468)
(1, 2459)
(1, 2453)
(1, 2448)
(1, 2442)
(1, 2435)
(1, 2430)
(1, 2425)
(1, 2417)
(1, 2413)
(1, 2406)
(1, 2402)
(1, 2394)
(1, 2389)
(1, 2382)
(1, 2375)
(1, 2372)
(1, 2363)
(1, 2357)
(1, 2353)
(1, 2345)
(1, 2337)
(1, 2333)
(1, 2330)
(1, 2319)
(1, 2315)
(1, 2309)
(1, 2304)
(1, 2296)
(1, 2290)
(1, 2284)
(1, 2279)
(1, 2273)
(1, 2267)
(1, 2260)
(1, 2254)
(1, 2249)
(1, 2244)
(1, 2236)
(1, 2231)
(1, 2225)
(1, 2219)
(1, 2213)
(1, 2207)
(1, 2202)
(1, 2193)
(1, 2192)
(1, 2185)
(1, 2175)
(1, 2172)
(1, 2160)
(1, 2158)
(1, 2150)
(1, 2144)
(1, 2137)
(1, 2136)
(1, 2128)
(1, 2126)
(1, 2116)
(1, 2110)
(1, 2106)
(1, 2105)
(1, 2094)
(1, 2088)
(1, 2087)
(1, 2077)
(1, 2070)
(1, 2065)
(1, 2058)
(1, 2052)
(1, 2048)
(1, 2040)
(1, 2035)
(1, 2026)
(1, 2023)
(1, 2016)
(1, 2012)
(1, 2005)
(1, 2000)
(1, 1992)
(1, 1989)
(1, 1983)
(1, 1976)
(1, 1970)


(2, 85)
(2, 78)
(2, 74)
(2, 71)
(2, 66)
(2, 62)
(2, 56)
(2, 54)
(2, 49)
(2, 45)
(2, 41)
(2, 37)
(2, 33)
(2, 28)
(2, 25)
(2, 23)
(2, 20)
(2, 18)
(2, 16)
(2, 14)
(2, 12)
(2, 10)
(2, 9)
(2, 6)
(2, 5)
(2, 3)
(2, 1)
(3, 2643)
(3, 2641)
(3, 2637)
(3, 2634)
(3, 2633)
(3, 2628)
(3, 2625)
(3, 2619)
(3, 2616)
(3, 2613)
(3, 2608)
(3, 2605)
(3, 2598)
(3, 2596)
(3, 2590)
(3, 2588)
(3, 2583)
(3, 2580)
(3, 2578)
(3, 2572)
(3, 2567)
(3, 2557)
(3, 2551)
(3, 2544)
(3, 2542)
(3, 2536)
(3, 2531)
(3, 2524)
(3, 2518)
(3, 2513)
(3, 2504)
(3, 2501)
(3, 2498)
(3, 2488)
(3, 2487)
(3, 2477)
(3, 2471)
(3, 2466)
(3, 2464)
(3, 2456)
(3, 2449)
(3, 2444)
(3, 2437)
(3, 2432)
(3, 2426)
(3, 2420)
(3, 2414)
(3, 2407)
(3, 2401)
(3, 2395)
(3, 2390)
(3, 2385)
(3, 2377)
(3, 2370)
(3, 2365)
(3, 2361)
(3, 2350)
(3, 2348)
(3, 2338)
(3, 2336)
(3, 2326)
(3, 2321)
(3, 2320)
(3, 2313)
(3, 2307)
(3, 2300)
(3, 2294)
(3, 2288)
(3, 2282)
(3, 2278)
(3, 2271)
(3, 2265)
(3, 2259)
(3, 2251)
(3, 2247)
(3, 2241)
(3, 2235)
(3, 2230)
(3, 2218)

(4, 530)
(4, 524)
(4, 518)
(4, 513)
(4, 508)
(4, 503)
(4, 497)
(4, 490)
(4, 486)
(4, 484)
(4, 474)
(4, 467)
(4, 461)
(4, 455)
(4, 449)
(4, 444)
(4, 437)
(4, 432)
(4, 431)
(4, 421)
(4, 415)
(4, 410)
(4, 404)
(4, 397)
(4, 392)
(4, 384)
(4, 380)
(4, 374)
(4, 368)
(4, 361)
(4, 356)
(4, 354)
(4, 344)
(4, 343)
(4, 338)
(4, 331)
(4, 326)
(4, 320)
(4, 319)
(4, 308)
(4, 302)
(4, 296)
(4, 289)
(4, 284)
(4, 278)
(4, 271)
(4, 266)
(4, 264)
(4, 254)
(4, 248)
(4, 241)
(4, 235)
(4, 231)
(4, 225)
(4, 218)
(4, 212)
(4, 206)
(4, 205)
(4, 195)
(4, 188)
(4, 182)
(4, 178)
(4, 175)
(4, 164)
(4, 160)
(4, 153)
(4, 147)
(4, 143)
(4, 136)
(4, 129)
(4, 122)
(4, 118)
(4, 112)
(4, 108)
(4, 101)
(4, 94)
(4, 86)
(4, 80)
(4, 76)
(4, 73)
(4, 69)
(4, 65)
(4, 61)
(4, 60)
(4, 53)
(4, 48)
(4, 44)
(4, 40)
(4, 36)
(4, 32)
(4, 31)
(4, 24)
(5, 2680)
(5, 2679)
(5, 2676)
(5, 2674)
(5, 2673)
(5, 2671)
(5, 2669)
(5, 2667)
(5, 2665)
(5, 2662)
(5, 2661)
(5, 2659)
(5, 2657)
(5, 2655)
(5, 2653)
(5, 2649)
(5, 2647)
(5, 2645)
(5, 2640)

(6, 769)
(6, 762)
(6, 761)
(6, 749)
(6, 744)
(6, 736)
(6, 732)
(6, 726)
(6, 719)
(6, 713)
(6, 708)
(6, 706)
(6, 695)
(6, 689)
(6, 685)
(6, 679)
(6, 673)
(6, 667)
(6, 659)
(6, 657)
(6, 651)
(6, 642)
(6, 637)
(6, 630)
(6, 625)
(6, 624)
(6, 612)
(6, 610)
(6, 601)
(6, 591)
(6, 582)
(6, 579)
(6, 574)
(6, 566)
(6, 559)
(6, 553)
(6, 546)
(6, 538)
(6, 532)
(6, 526)
(6, 516)
(6, 510)
(6, 504)
(6, 498)
(6, 493)
(6, 487)
(6, 480)
(6, 475)
(6, 470)
(6, 463)
(6, 457)
(6, 451)
(6, 441)
(6, 439)
(6, 428)
(6, 426)
(6, 419)
(6, 412)
(6, 406)
(6, 400)
(6, 393)
(6, 386)
(6, 377)
(6, 370)
(6, 364)
(6, 359)
(6, 352)
(6, 347)
(6, 340)
(6, 335)
(6, 328)
(6, 322)
(6, 315)
(6, 309)
(6, 304)
(6, 297)
(6, 293)
(6, 285)
(6, 279)
(6, 273)
(6, 267)
(6, 260)
(6, 257)
(6, 250)
(6, 245)
(6, 238)
(6, 233)
(6, 228)
(6, 220)
(6, 216)
(6, 209)
(6, 203)
(6, 198)
(6, 192)
(6, 189)
(6, 179)
(6, 174)
(6, 168)
(6, 161)
(6, 157)
(6, 148)
(6, 144)
(6, 137)
(6, 132)
(6, 130)
(6, 123)
(6, 114)
(6, 113)
(6, 102)
(6, 97)
(6, 90)
(6,